In [ ]:
#we use None for the blank 
class Tape:
  def __init__(self,left,current,right):
    self.left = left
    self.current = current
    self.right = right

  def write(self,c):
    self.current = c

  def move(self,m):
    if m == 'L':
      if self.left == []:
          self.right.insert(0,self.current)
          self.current = None
      else:
          self.right.insert(0,self.current)
          self.current = self.left.pop()
    elif m == 'R':
      if self.right == []:
          self.left.append(self.current)
          self.current = None
      else:
          self.left.append(self.current)
          self.current = self.right[0]
          del self.right[0]

  def act(self,c,m):
    print(c,m)
    self.write(c)
    self.move(m)

  def show(self):
    print(self.left,self.current,self.right)

In [ ]:
#Esempi
tape = Tape([],'0',[])
while True:
  action = input()
  c,m = action.split(",")
  tape.act(c,m)
  tape.show()

In [ ]:
class Configuration:
  def __init__(self,tape,state):
    self.tape = tape
    self.state = state

In [ ]:
class TuringMachine:
  def __init__(self,Sigma,Q,start,halt,code):
    self.Sigma = Sigma
    self.states = Q
    self.start = start
    self.halt = halt
    self.code = code

  def action(self,q,a):
    print("act:",q,a)
    for (q1,a1,q2,a2,m) in self.code:
      if q1==q and a1==a:
        return (q2,a2,m)
    return None

In [ ]:
def step(c,M):
  a = c.tape.current
  s = c.state
  action = M.action(s,a)
  #print("action is ",action)
  if action:
    qnew,anew,m = action
    c.state = qnew
    c.tape.act(anew,m)
  else: #no action found
    c.state = 'error' #we suppose to always have a special error state

def exec(c,M):
  while not(c.state=='error') and not(M.halt(c.state)):
    step(c,M)
    c.tape.show()

In [ ]:
#example
#una macchina che si sposta a destra fino al primo blank

goright = TuringMachine(['0','1'],
                        ['start','end'],
                        'start',
                        lambda s: s=='end',
                        [('start','0','start','0','R'),
                         ('start','1','start','1','R'),
                         ('start',None,'end',None,'L')])

init_configuration = Configuration(Tape([],'0',['0','1','0']),goright.start)
exec(init_configuration,goright)

act: start 0
0 R
['0'] 0 ['1', '0']
act: start 0
0 R
['0', '0'] 1 ['0']
act: start 1
1 R
['0', '0', '1'] 0 []
act: start 0
0 R
['0', '0', '1', '0'] None []
act: start None
None L
['0', '0', '1'] 0 [None]


In [ ]:
# riconoscimento del linguaggio delle stringe w#w dove w è una stringa di 0 e 1.
# marchiano i caratteri letti con * e quelli corrispondenti nella seconda stringa con $
# il nastro è tipicamente in uno stato della forma
#        **..*w#$$..$w 
# dove il numero di $ e uguale o inferiore di uno a quello delle *, a secondo che il
# carattere letto sia già stato "matchato" oppure no.
# Il carrattere letto x è salvato nello stato interno.
# La macchina si trova in un di qusto stati:
# - start
# - cerco x, sono a sinistra di #
# - cerco x, sono a destra di #
# - ritorno
# - accetta
# - rifiuta

In [ ]:
mach = TuringMachine(['0','1','#','*','$'],
                     ['start',
                      'cerco 0 sono a sinistra di #',
                      'cerco 1 sono a sinistra di #',
                      'cerco 0 sono a destra di #',
                      'cerco 1 sono a destra di #',
                      'ritorno',
                      'accetta',
                      'rifiuta'
                      ],
                     'start',
                     lambda s: s == 'accetta' or s == 'rifiuta',
                     [('start','0','cerco 0 sono a sinistra di #','*','R'),
                      ('start','1','cerco 1 sono a sinistra di #','*','R'),
                      ('start','#','cerco None sono a destra di #','#','R'),
                      ('cerco 0 sono a sinistra di #','0','cerco 0 sono a sinistra di #','0','R'), 
                      ('cerco 0 sono a sinistra di #','1','cerco 0 sono a sinistra di #','1','R'),
                      ('cerco 0 sono a sinistra di #','#','cerco 0 sono a destra di #','#','R'),
                      ('cerco 1 sono a sinistra di #','0','cerco 1 sono a sinistra di #','0','R'),
                      ('cerco 1 sono a sinistra di #','1','cerco 1 sono a sinistra di #','1','R'),
                      ('cerco 1 sono a sinistra di #','#','cerco 1 sono a destra di #','#','R'),
                      ('cerco 0 sono a destra di #','0','ritorno','$','L'),
                      ('cerco 0 sono a destra di #','$','cerco 0 sono a destra di #','$','R'),
                      ('cerco 1 sono a destra di #','1','ritorno','$','L'),
                      ('cerco 1 sono a destra di #','$','cerco 1 sono a destra di #','$','R'),
                      ('cerco None sono a destra di #', None,'accept', None,'R'),
                      ('cerco None sono a destra di #', '$','cerco None sono a destra di #', '$','R'),
                      ('ritorno','$','ritorno','$','L'),
                      ('ritorno','#','ritorno','#','L'),
                      ('ritorno','0','ritorno','0','L'),
                      ('ritorno','1','ritorno','1','L'),
                      ('ritorno','*','start','*','R')
                      ])

In [ ]:
conf = Configuration(Tape([],'0',['0','1','#','0','0','1']),mach.start)

In [ ]:
exec(conf,mach)

act: start 0
* R
['*'] 0 ['1', '#', '0', '0', '1']
act: cerco 0 sono a sinistra di # 0
0 R
['*', '0'] 1 ['#', '0', '0', '1']
act: cerco 0 sono a sinistra di # 1
1 R
['*', '0', '1'] # ['0', '0', '1']
act: cerco 0 sono a sinistra di # #
# R
['*', '0', '1', '#'] 0 ['0', '1']
act: cerco 0 sono a destra di # 0
$ L
['*', '0', '1'] # ['$', '0', '1']
act: ritorno #
# L
['*', '0'] 1 ['#', '$', '0', '1']
act: ritorno 1
1 L
['*'] 0 ['1', '#', '$', '0', '1']
act: ritorno 0
0 L
[] * ['0', '1', '#', '$', '0', '1']
act: ritorno *
* R
['*'] 0 ['1', '#', '$', '0', '1']
act: start 0
* R
['*', '*'] 1 ['#', '$', '0', '1']
act: cerco 0 sono a sinistra di # 1
1 R
['*', '*', '1'] # ['$', '0', '1']
act: cerco 0 sono a sinistra di # #
# R
['*', '*', '1', '#'] $ ['0', '1']
act: cerco 0 sono a destra di # $
$ R
['*', '*', '1', '#', '$'] 0 ['1']
act: cerco 0 sono a destra di # 0
$ L
['*', '*', '1', '#'] $ ['$', '1']
act: ritorno $
$ L
['*', '*', '1'] # ['$', '$', '1']
act: ritorno #
# L
['*', '*'] 1 ['#', '$', '$

aaa